In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install h5py
!pip install typing-extensions
!pip install wheel
!sudo add-apt-repository ppa:mc3man/trusty-media
!sudo apt-get update
!sudo apt-get install ffmpeg
!pip install unidecode
!pip install basketball-reference-scraper
!pip3 install imageio-ffmpeg
!pip install --upgrade imageio-ffmpeg
!sudo pip3 install numpy --upgrade
!sudo apt install ffmpeg
!pip install moviepy==1.0.0
!sudo pip3 install imageio==2.4.1 --user
!pip install "numpy>=1.16.5,<1.23.0"
!conda install "numpy>=1.16.5,<1.23.0"
!pip install youtube_dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 Upgraded, advanced or not normally available multimedia packages for Trusty
Xenial users go here - https://launchpad.net/~mc3man/+archive/ubuntu/xerus-media

*Please note that if using this ppa I would *not* try upgrading to 14.10/15.04, ect. Do a fresh install instead. The intent here is just for users wishing to stay on 14.04*

If upgrading releases anyway use ppa-purge *First* -
sudo ppa-purge  ppa:mc3man/trusty-media

Also note that using this ppa then disabling may cause issue for installing i386 packages like used by wine. So once enabled leave enabled or purge before removing.

Additionally if using apt-get * sudo  apt-get dist-upgrade will be needed* at times.(pay attention).  Otherwis

In [ ]:
from __future__ import unicode_literals
import youtube_dl
ydl_opts = {}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=8MU1owTNV10']) #動画URL

In [ ]:
from pipes import Template
import cv2
import numpy as np
import glob
from moviepy.editor import *

def match_template(template, frame):
  # 処理対象画像に対して、テンプレート画像との類似度を算出する
  res = cv2.matchTemplate(frame, template, cv2.TM_CCOEFF_NORMED)

  # 類似度の高い部分を検出する
  threshold = 0.8
  loc = np.where(res >= threshold)

  ret = []
  # 検出した部分
  for pt in zip(*loc[::-1]):
    ret.append(pt)

  return ret

def crop_movie(src_file_path, start, end, fps, export_file_path):
  video = VideoFileClip(src_file_path).subclip(start, end) #ビデオのカット開始
  video.write_videofile(export_file_path,
    codec='libx264',
    audio_codec='aac',
    temp_audiofile='temp-audio.m4a',
    fps=fps,
    remove_temp=True
  )

if __name__ == '__main__':
  movie_file_dir =  '/content/drive/MyDrive/課題/movie/' #プレイ元動画格納ファイル
  export_file_dir = '/content/drive/MyDrive/課題/kill_log/' #killclip格納ファイル

  movie_list = glob.glob(movie_file_dir + '*.mp4') #プレイ元動画
  
  template = cv2.imread('/content/drive/MyDrive/課題/template.png') #テンプレート画像

  for movie in movie_list:
    cap = cv2.VideoCapture(movie)
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fc = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)//1
    frameidx = 0
    taoshita = False
    start = 0.0
    end = 0.0
    frame_skip = 30
    movie_idx = 0
    eval = 0

    while True:
      ret, frame = cap.read()
      if ret:
        locations = match_template(template, frame)
        second = float(frameidx)/fps
        if(len(locations) > 0):
          cv2.imwrite('frame.jpg', frame)

          if(taoshita == False):
            taoshita = True
            start = second
            eval = len(locations)
        else:
          if(taoshita == True):
            taoshita = False
            end = second

            export_filename = export_file_dir + str(movie_idx)+'_'+ movie
            crop_movie(movie, int(start)-1, int(end), fps, export_filename)
            movie_idx = movie_idx + 1
            
      else:
        break
      frameidx = frameidx + 1
      
      for i in range(0, frame_skip):
        ret, frame = cap.read()
        if ret:
            pass
        else:
            break
        frameidx = frameidx + 1

    cap.release()

  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



In [5]:
import cv2
import glob

def comb_movie(movie_files,out_path):

    # 形式はmp4
    fourcc = cv2.VideoWriter_fourcc('m','p','4','v')

    # 動画情報の取得
    movie = cv2.VideoCapture(movie_files[0])
    fps = movie.get(cv2.CAP_PROP_FPS)
    height = movie.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = movie.get(cv2.CAP_PROP_FRAME_WIDTH)


    # 出力先のファイルを開く
    out = cv2.VideoWriter(out_path, int(fourcc), fps, (int(width), int(height)))


    for movies in (movie_files):
        print(movies)
        # 動画ファイルの読み込み，引数はビデオファイルのパス
        movie = cv2.VideoCapture(movies)

        # 正常に動画ファイルを読み込めたか確認
        if movie.isOpened() == True: 
            # read():1コマ分のキャプチャ画像データを読み込む
            ret, frame = movie.read() 
        else:
            ret = False

        while ret:
            # 読み込んだフレームを書き込み
            out.write(frame)
            # 次のフレーム読み込み
            ret, frame = movie.read()

# ディレクトリ内の動画をリストで取り出す
files = sorted(glob.glob("/content/drive/MyDrive/課題/kill_log/*.mp4"))

# 出力ファイル名
out_path = "movie_out1.mp4"

comb_movie(files,out_path)

/content/drive/MyDrive/課題/kill_log/0_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/10_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/11_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/12_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/13_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/14_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/15_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/16_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/17_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/18_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/1_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/2_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/3_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/4_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/5_E_valorant_2023_01_20.mp4
/content/drive/MyDrive/課題/kill_log/6_E_valoran